## AoC 2023 - Day xx

### Setup imports and get raw data

In [90]:
from collections import deque
from icecream import ic

#grid = {complex(x,y): val for y, row in enumerate(data) for x, val in enumerate(row)}
#grid = open('input.txt').read().split('\n')
grid = open('test.txt').read().split('\n')

limits = (len(grid[0]) - 1, len(grid) - 1)

start = (0, grid[0].index("."))
end = (len(grid) - 1, grid[-1].index("."))

#dirs = [((1), '.^'), (-1, '.v'), (1j, '.>'), (-1j, '.<')]  # north, south, east, west
#              Up             Down            Left             Right
directions = (((-1, 0),'.^'), ((1, 0), '.v'), ((0, -1), '.<'), ((0, 1), '.>'))

_ = ic(start, end, directions, limits)

def print_grid(grid, all_visited):
    for y,line in enumerate(grid):
        l = ''
        #print(y)
        for x,char in enumerate(line):
            #ic(x, y)
            if grid[y][x] in '.^v<>':
                if (y, x) in all_visited:
                    l += 'O'
                else:
                    l += grid[y][x]
            else:
                l += "#"
        print(l)

ic| start: (0, 1)
    end: (22, 21)
    directions: (((-1, 0), '.^'), ((1, 0), '.v'), ((0, -1), '.<'), ((0, 1), '.>'))
    limits: (22, 22)


In [69]:
all_visited = set()
all_visited.add(start)

queue = deque([(start, set([(start)]))])

routes = []
#while queue:
while queue:
    (pos_x, pos_y), visited = queue.popleft()
    
    #pos_x, pos_y = pos
    
    for (dir_x, dir_y), valid in directions:
        look_x, look_y = pos_x + dir_x, pos_y + dir_y
        
        if (look_x, look_y) == end:
            routes.append(len(visited))
            
        inbounds = (0 <= look_x <= limits[0]) and (0 <= look_y <= limits[1])
        if inbounds and grid[look_x][look_y] in '<>^v':
            all_visited.add((look_x, look_y))
            queue.append(((look_x, look_y), visited))
        elif inbounds and grid[look_x][look_y] in valid and (look_x, look_y) not in visited:
            
            visited.add((look_x, look_y))
            all_visited.add((look_x, look_y))
            queue.append(((look_x, look_y), visited))
            

#ic(queue)
#ic(all_visited)


_ = ic(routes)

ic| routes: [190]


### HN's solution
It's a DFS, but modified to include the number of steps between "nodes"; spots on the map where you can take 2 or more paths (not including the one you came from).

Once you have that graph, it's relatively easy to work out which path is best, always taking the **longest** route

In [98]:
grid = open('input.txt').read().split('\n')
start = (0, grid[0].index("."))
end = (len(grid) - 1, grid[-1].index("."))
points = [start, end]

for r, row in enumerate(grid):
    for c, ch in enumerate(row):
        if ch == "#":
            continue
        neighbors = 0
        for nr, nc in [(r - 1, c), (r + 1, c), (r, c - 1), (r, c + 1)]:
            if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]) and grid[nr][nc] != "#":
                neighbors += 1
        if neighbors >= 3:
            points.append((r, c))

graph = {pt: {} for pt in points}

dirs = {
    "^": [(-1, 0)],
    "v": [(1, 0)],
    "<": [(0, -1)],
    ">": [(0, 1)],
    ".": [(-1, 0), (1, 0), (0, -1), (0, 1)],
}

for sr, sc in points:
    stack = [(0, sr, sc)]
    seen = {(sr, sc)}

    while stack:
        n, r, c = stack.pop()
        
        if n != 0 and (r, c) in points:
            graph[(sr, sc)][(r, c)] = n
            continue

        for dr, dc in dirs[grid[r][c]]:
            nr = r + dr
            nc = c + dc
            if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]) and grid[nr][nc] != "#" and (nr, nc) not in seen:
                stack.append((n + 1, nr, nc))
                seen.add((nr, nc))
                
#_ = ic(graph)

graph_p2 = {pt: {} for pt in points}

for sr, sc in points:
    stack = [(0, sr, sc)]
    seen = {(sr, sc)}

    while stack:
        n, r, c = stack.pop()
        
        if n != 0 and (r, c) in points:
            graph_p2[(sr, sc)][(r, c)] = n
            continue

        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr = r + dr
            nc = c + dc
            if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]) and grid[nr][nc] != "#" and (nr, nc) not in seen:
                stack.append((n + 1, nr, nc))
                seen.add((nr, nc))

#_ = ic(graph_p2)

def dfs(pt, g, s):
    if pt == end:
        return 0

    m = -float("inf")

    s.add(pt)
    for nx in g[pt]:
        if nx not in s:
            m = max(m, dfs(nx, g, s) + g[pt][nx])
    s.remove(pt)

    return m

ic(dfs(start, graph, set()))
ic(dfs(start, graph_p2, set()))

ic| dfs(start, graph, set()): 2406
ic| dfs(start, graph_p2, set()): 6630


6630